In [27]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [28]:
from crewai import Agent, Task, Crew

In [29]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, FirecrawlCrawlWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
linkedin = ScrapeWebsiteTool(
    website_url='https://www.linkedin.com/',
)
github = ScrapeWebsiteTool(
    website_url='https://github.com/',
)
medium = ScrapeWebsiteTool(
    website_url='https://medium.com/',
)
#firecrawl = FirecrawlCrawlWebsiteTool('https://www.google.com/')

In [30]:
import os
from utils import get_groc_api_key

groc_api_key = get_groc_api_key()
#os.environ["GROQ_API_KEY"] = groc_api_key
os.environ["GROQ_API_KEY"] = 'deepseek-r1-distill-llama-70b'

In [31]:
# Create the LLM configuration
llm_config = {
    "api_key": groc_api_key,
    "model": "deepseek-r1-distill-llama-70b",
    "temperature": 0.2,
    "max_tokens": 500
}


In [51]:
# Update initial input and run (replace existing run command)
user_details = {
    "user_name": "Gayashan Wagachchi",
    "email": "gayashan4lk@gmail.com",
    "educational_background": "Bsc in Science from University of Colombo",
    "professional_background": "Software Engineer | Full stack developer | Tech podcaster",
    "skills": ["HTML", "CSS", "Python", "Node.js"],
    "linkedin": "https://www.linkedin.com/in/gayashan4lk/",
    "github": "https://github.com/gayashan4lk",
    "medium": "https://medium.com/@gayashan4lk"
}

In [52]:
# Define Data Collector Agent
profile_finder = Agent(
    role="User Information Collection Specialist",
    goal=f"Gather accurate and comprehensive initial user information for {user_details['user_name']} through web form interface",
    backstory="You're an expert data collection specialist with years of experience in user onboarding and "
              "information gathering. Your meticulous attention to detail ensures that all essential user "
              "information is captured correctly the first time. You understand the importance of building "
              "user trust through transparent data collection practices and clear communication.",
    instructions=[
        f"1. Present a clean, user-friendly interface asking for {user_details['user_name']}'s name, {user_details['email']}, {user_details['educational_background']} , {user_details['professional_background']} , and skil{user_details['skills']}s.",
        "2. Validate all input fields for proper formatting and completeness.",
        "3. Clearly explain how the collected information will be used.", 
        "4. Store the collected information securely for use by other agents.",
        "5. Ensure all information is properly structured for downstream analysis."
    ],
    tools=[search_tool, scrape_tool],
    allow_delegation=False,
    verbose=True
)

In [53]:
web_researcher = Agent(
    role="General Web Research Specialist",
    goal=f"Discover and collect all publicly available information about {user_details['user_name']} through general web searches",
    backstory="You're a digital detective with unparalleled skills in uncovering information across the internet. "
              "With your background in OSINT (Open Source Intelligence), you've developed techniques to efficiently "
              "filter through vast amounts of online data to find relevant information about individuals. Your "
              "research provides the foundational context that other specialists build upon.",
    instructions=[
        f"1. Conduct comprehensive web searches based on {user_details['user_name']}'s information.",
        "2. Focus searches on professional, educational, and skill-related information.",
        "3. Prioritize credible sources and verify information across multiple sites when possible.",
        "4. Structure the collected information in a standardized format.",
        "5. Flag any inconsistencies or contradictions in discovered information.",
        "6. Provide confidence scores for each piece of information discovered."
    ],
    tools=[search_tool, scrape_tool],
    allow_delegation=False,
    verbose=True
)

In [54]:
linkedin_specialist = Agent(
    role="LinkedIn Profile Analysis Expert",
    goal=f"Extract and analyze complete professional information from {user_details['user_name']}'s LinkedIn presence at {user_details['linkedin']}",
    backstory="You've spent your career analyzing professional networks and digital career footprints. Having worked "
              "in talent acquisition and professional network analysis, you understand how to interpret LinkedIn "
              "profiles to identify career trajectories, skill development patterns, and professional accomplishments. "
              "Your expertise helps create comprehensive professional profiles beyond what's immediately visible.",
    instructions=[
        f"1. Access {user_details['user_name']}'s LinkedIn profile at {user_details['linkedin']} using appropriate scraping tools or APIs.",
        f"2. Extract {user_details['user_name']}'s complete work history, including job titles, companies, durations, and responsibilities.",
        f"3. Collect {user_details['user_name']}'s all educational credentials including degrees, institutions, and graduation dates.",
        f"4. Gather {user_details['user_name']}'s certifications, projects, research publications, and other professional achievements.",
        f"5. Analyze {user_details['user_name']}'s connections and endorsements to identify key skill areas.",
        "6. Document any articles, posts, or comments that demonstrate thought leadership.",
        "7. Organize the information chronologically and by relevance to create a comprehensive professional narrative."
    ],
    tools=[linkedin, scrape_tool],
    allow_delegation=False,
    verbose=True
)


In [55]:
github_analyst = Agent(
    role="GitHub Repository and Code Analysis Specialist",
    goal=f"Analyze {user_details['user_name']}'s GitHub presence at {user_details['github']} to determine technical capabilities, coding patterns, and project history",
    backstory="As a veteran software engineer and code reviewer, you've developed a keen eye for recognizing coding styles, "
              "technical capabilities, and development patterns. Having reviewed thousands of repositories across different "
              "domains, you can quickly assess a developer's technical proficiency, preferred technologies, and coding best "
              "practices through their GitHub contributions.",
    instructions=[
        f"1. Access the {user_details['github']}'s GitHub profile and all public repositories.",
        f"2. Analyze {user_details['github']}'s repository metadata (stars, forks, watchers, issues, pull requests).",
        f"3. Identify {user_details['github']}'s primary programming languages and technology stacks used.",
        f"4. Evaluate {user_details['github']}'s code quality, documentation practices, and adherence to best practices.",
        f"5. Assess {user_details['github']}'s contribution frequency, commit patterns, and project maintenance.",
        "6. Analyze README files and documentation for communication clarity.",
        "7. Review issues and pull requests for collaboration style and problem-solving approach.",
        "8. Identify key projects that demonstrate technical expertise and development philosophy."
    ],
    tools=[github, scrape_tool],
    allow_delegation=False,
    verbose=True
)

In [56]:
content_analyst = Agent(
    role="Medium Content and Technical Writing Expert",
    goal=f"Discover and analyze {user_details['user_name']}'s written content to identify expertise areas and thought leadership",
    backstory="You've built your career analyzing technical content and identifying subject matter expertise from "
              "written work. With a background in content strategy and technical publishing, you can quickly identify "
              "an author's knowledge domains, communication style, and thought leadership based on their published "
              "articles. Your analysis reveals expertise that might not be explicitly stated in professional profiles.",
    instructions=[
        f"1. Locate all Medium articles and blog posts authored by {user_details['user_name']} at {user_details['medium']}.",
        f"2. Analyze {user_details['user_name']} article topics, categories, and tags to identify primary focus areas.",
        "3. Assess writing style, technical depth, and communication clarity.",
        "4. Identify recurring themes and expertise domains across multiple articles.",
        "5. Evaluate audience engagement metrics (claps, comments, shares).",
        "6. Extract key technologies, methodologies, or frameworks mentioned in articles.",
        "7. Create a content timeline to track evolution of expertise and interests.",
        "8. Compare content focus areas with stated skills and professional background."
    ],
    tools=[medium, scrape_tool],
    allow_delegation=False,
    verbose=True
)


In [57]:
data_synthesizer = Agent(
    role="Cross-Source Data Integration Analyst",
    goal=f"Integrate and analyze all collected user data from {profile_finder.role}, {web_researcher.role}, {linkedin_specialist.role}, {github_analyst.role}, and {content_analyst.role} to create a unified, validated user profile",
    backstory="You're a data synthesis expert with extensive experience in cross-referencing information from "
              "multiple sources to create cohesive user profiles. Your background in data science and pattern "
              "recognition allows you to identify connections between disparate pieces of information and resolve "
              "conflicts between data sources. Your work transforms raw data collections into meaningful insights "
              "about individuals.",
    instructions=[
        f"1. Consolidate all information collected by {profile_finder.role}, {web_researcher.role}, {linkedin_specialist.role}, {github_analyst.role}, and {content_analyst.role}.",
        "2. Cross-reference data points across sources to validate accuracy and resolve contradictions.",
        "3. Identify patterns and connections between professional history, technical skills, and content creation.",
        "4. Create a timeline of skill development and career progression.",
        "5. Detect any gaps or inconsistencies that require further investigation.",
        "6. Assign confidence scores to each consolidated data point.",
        "7. Generate insights about the user's expertise evolution, skill specializations, and professional focus areas.",
        "8. Prepare a comprehensive profile with validated information ready for structured formatting by the persona generator."
    ],
    allow_delegation=False,
    verbose=True
)

In [58]:
persona_generator = Agent(
    role="User Persona Creation and JSON Formatting Specialist",
    goal=f"Generate a comprehensive, structured JSON representation of {user_details['user_name']}'s complete professional persona by integrating data from all sources",
    backstory="You are a data synthesis expert specializing in creating structured, machine-readable formats "
              "from complex user information. With expertise in data architecture and profiling, you excel at "
              "organizing disparate information into cohesive JSON structures that capture the complete "
              "professional identity of individuals.",
    instructions=[
        f"1. Integrate and validate data from {web_researcher.role}, {linkedin_specialist.role}, {github_analyst.role}, and {content_analyst.role}",
        f"2. Cross-reference information collected by {profile_finder.role} with discovered data",
        "3. Structure all validated information into a standardized JSON schema",
        "4. Identify and organize categories: Personal Info, Education, Experience, Skills, Projects, Content",
        "5. Ensure proper nesting and relationships between data elements",
        "6. Include confidence scores for data points",
        "7. Document any conflicts between sources",
        "8. Generate metadata about the information gathering process",
        f"9. Synthesize insights from {data_synthesizer.role}'s analysis"
    ],
    tools=[search_tool, scrape_tool],
    allow_delegation=False,
    verbose=True,
    llm_config=llm_config
)

# tasks

In [59]:
task_initial_data_collection = Task(
    description=(
        "1. Present a clean, user-friendly form interface for data collection.\n"
        "2. Collect and validate user's full name, email address, educational background.\n"
        "3. Gather professional background summary and key skills.\n"
        "4. Validate all input fields for proper formatting.\n"
        "5. Explain data usage transparently to build user trust.\n"
        "6. Format and sanitize collected data for downstream processing."
    ),
    expected_output=(
        "A structured dictionary containing:\n"
        "- First and last name (as separate fields)\n"
        "- Validated email address\n"
        "- Educational background summary\n"
        "- Professional background summary\n"
        "- Self-reported skills list (array format)\n"
        "- All fields properly sanitized and formatted"
    ),
    tools=[search_tool, scrape_tool],
    agent=profile_finder
)

In [60]:
task_web_search = Task(
    description=(
        "1. Conduct comprehensive web searches about the user combining name with professional keywords.\n"
        "2. Discover professional profiles, academic publications, and conference appearances.\n"
        "3. Find company mentions, news articles, and other relevant information.\n"
        "4. Cross-reference findings with user-provided details for validation.\n"
        "5. Document all discovered web mentions and their sources.\n"
        "6. Evaluate confidence levels for each piece of information found."
    ),
    expected_output=(
        "A detailed report containing:\n"
        "- Web mentions categorized by source type (news, academic, professional)\n"
        "- Links to relevant webpages with user mentions\n"
        "- Summary of discovered education, professional history, and skills\n"
        "- Confidence scores for each information piece\n"
        "- Discrepancies between user-provided and discovered information\n"
        "- Additional information sources for further investigation"
    ),
    tools=[search_tool, scrape_tool],
    agent=web_researcher,
    dependencies=[task_initial_data_collection]
)

In [61]:
task_linkedin_analysis = Task(
    description=(
        "1. Access and analyze the user's LinkedIn profile using appropriate scraping tools.\n"
        "2. Extract complete work history with positions, companies, and durations.\n"
        "3. Collect educational credentials and certifications.\n"
        "4. Gather skills, endorsements, and recommendations.\n"
        "5. Extract projects, publications, patents, and professional work.\n"
        "6. Analyze connection patterns and industry distribution.\n"
        "7. Collect authored content including articles and posts.\n"
        "8. Document awards, honors, and recognitions."
    ),
    expected_output=(
        "A comprehensive LinkedIn analysis containing:\n"
        "- Chronological work history with company details\n"
        "- Educational credentials and certifications\n"
        "- Skills and endorsements with metrics\n"
        "- Projects and publications\n"
        "- Professional recommendations\n"
        "- Authored content analysis\n"
        "- Network analysis and industry focus\n"
        "- Professional achievements and awards"
    ),
    tools=[search_tool, scrape_tool],
    agent=linkedin_specialist,
    dependencies=[task_initial_data_collection]
)

In [62]:
task_github_analysis = Task(
    description=(
        "1. Access and analyze all public repositories in user's GitHub profile.\n"
        "2. Extract repository metadata including stars, forks, watchers, and contributors.\n"
        "3. Identify primary programming languages and technology stacks with usage percentages.\n"
        "4. Evaluate code quality, organization, documentation, and best practices.\n"
        "5. Analyze contribution patterns including frequency and time distribution.\n"
        "6. Review collaboration patterns from issues, pull requests, and code reviews.\n"
        "7. Categorize projects by type and domain focus.\n"
        "8. Document notable technical achievements and innovative solutions.\n"
        "9. Track evolution of technical skills based on repository timeline."
    ),
    expected_output=(
        "A comprehensive GitHub analysis containing:\n"
        "- Repository list with complete metadata\n"
        "- Technology stack analysis with usage metrics\n"
        "- Code quality assessment report\n"
        "- Contribution pattern analysis\n"
        "- Collaboration style evaluation\n"
        "- Project categorization and domain expertise\n"
        "- Technical achievement highlights\n"
        "- Skill evolution timeline\n"
        "- Example code snippets demonstrating expertise"
    ),
    tools=[search_tool, scrape_tool],
    agent=github_analyst,
    dependencies=[task_initial_data_collection]
)

In [63]:
task_medium_analysis = Task(
    description=(
        "1. Discover and analyze all content published by the user on Medium and other platforms.\n"
        "2. Identify and catalog all articles authored by the user.\n"
        "3. Analyze topics, technical depth, and writing style of each article.\n"
        "4. Track audience engagement metrics where available.\n"
        "5. Identify recurring themes and technical focus areas.\n"
        "6. Assess thought leadership indicators and expertise evolution.\n"
        "7. Compare content focus with stated professional background.\n"
        "8. Document technologies and methodologies frequently discussed.\n"
        "9. Extract notable quotes demonstrating expertise.\n"
        "10. Create timeline of expertise development through content."
    ),
    expected_output=(
        "A comprehensive content analysis report containing:\n"
        "- Complete article catalog with metadata\n"
        "- Topic categorization and expertise domains\n"
        "- Technical depth assessment by domain\n"
        "- Writing style analysis and engagement metrics\n"
        "- Evolution of expertise timeline\n"
        "- Key technologies and frameworks coverage\n"
        "- Notable demonstrations of thought leadership\n"
        "- Alignment with professional background\n"
        "- Sources and confidence scores for findings"
    ),
    tools=[search_tool, scrape_tool],
    agent=content_analyst,
    dependencies=[task_initial_data_collection]
)

In [64]:
task_build_persona = Task(
    description=(
        "1. Collect and organize all analyzed data into defined categories.\n"
        "2. Structure information into Personal Information section.\n"
        "3. Organize Educational Background details chronologically.\n"
        "4. Format Professional Experience with company details and achievements.\n"
        "5. Categorize and rank Skills with proficiency levels.\n"
        "6. Document Coding Behavior patterns and preferences.\n"
        "7. Summarize Writing Topics and expertise areas.\n"
        "8. Convert the organized information into a structured JSON format."
    ),
    expected_output=(
        "A comprehensive JSON string containing:\n"
        "- Personal Information (name, contact, social profiles)\n"
        "- Educational Background with timeline\n"
        "- Professional Experience with detailed history\n"
        "- Skills categorized by domain and proficiency\n"
        "- Coding Behavior analysis and patterns\n"
        "- Writing Topics and thought leadership areas\n"
        "- All data properly formatted and nested"
    ),
    tools=[search_tool, scrape_tool],
    agent=persona_generator,
    dependencies=[task_initial_data_collection, task_web_search, task_linkedin_analysis, task_github_analysis, task_medium_analysis]
)

In [65]:
task_persona_generation = Task(
    description=(
        "1. Structure all collected and validated information into standardized JSON schema.\n"
        "2. Format personal information including name and contact details into standardized JSON schema.\n"
        "3. Organize educational experiences chronologically into standardized JSON schema.\n"
        "4. Structure professional experiences with detailed company information into standardized JSON schema.\n"
        "5. Categorize skills inventory with proficiency levels into standardized JSON schema.\n"
        "6. Document significant projects and technologies used into standardized JSON schema.\n"
        "7. Include publications and written works into standardized JSON schema.\n"
        "8. Add professional certifications and credentials into standardized JSON schema.\n"
        "9. Structure online presence data including GitHub and Medium profiles into standardized JSON schema.\n"
        "10. Categorize expertise areas with confidence levels into standardized JSON schema."
    ),
    expected_output=(
        "A complete JSON document containing:\n"
        "- Personal information (name, contact, profiles)\n"
        "- Educational background with timeline\n"
        "- Professional experience with detailed history\n"
        "- Categorized skills with proficiency levels\n"
        "- Projects and publications\n"
        "- Online presence analysis\n"
        f"- Expertise areas that took from the {user_details['github']} with confidence scores\n"
        "- Generation metadata and sources"
    ),
    tools=[search_tool, scrape_tool],
    agent=persona_generator,
    dependencies=[task_initial_data_collection, task_web_search, task_linkedin_analysis, 
                 task_github_analysis, task_medium_analysis, task_build_persona]
)

In [66]:
# Set up the crew (replace existing crew setup)
crew = Crew(
    agents=[profile_finder, web_researcher, linkedin_specialist, github_analyst, content_analyst, data_synthesizer, persona_generator],
    tasks=[task_initial_data_collection, task_web_search, task_linkedin_analysis, 
           task_github_analysis, task_medium_analysis, task_build_persona, task_persona_generation],
    verbose=True,
    #memory=True,
)

2025-03-15 22:06:13,478 - 33880 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [67]:
from IPython.display import Markdown as md

# Run the crew
result = crew.kickoff(user_details)
md(result.raw)

# Agent: User Information Collection Specialist
## Task: 1. Present a clean, user-friendly form interface for data collection.
2. Collect and validate user's full name, email address, educational background.
3. Gather professional background summary and key skills.
4. Validate all input fields for proper formatting.
5. Explain data usage transparently to build user trust.
6. Format and sanitize collected data for downstream processing.


# Agent: User Information Collection Specialist
## Thought: I need to gather the required information from Gayashan Wagachchi through a web form interface. The first step is to determine the best way to present the form and communicate about data usage to build trust. I will start by searching for best practices in user-friendly data collection forms.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"best practices for user-friendly data collection forms\"}"
## Tool Output: 
{'searchParameters': {'q': 'best practices 

```json
{
  "personal_information": {
    "name": {
      "first": "Gayashan",
      "last": "Wagachchi"
    },
    "email": {
      "address": "example@example.com",
      "validated": true
    },
    "social_profiles": {
      "linkedin": "https://lk.linkedin.com/in/gayashan4lk",
      "the_org": "https://theorg.com/org/bistec-global-services/org-chart/gayashan-wagachchi",
      "rocketreach": "https://rocketreach.co/gayashan-wagachchi-email_16665346"
    }
  },
  "educational_background": [
    {
      "degree": "Bachelor's degree",
      "field_of_study": "Computer Science",
      "institution": "University of Colombo",
      "duration": "2011 - 2015"
    },
    {
      "degree": "General Certificate of Education",
      "field_of_study": "A-Level (Biological Sciences)",
      "institution": "Rajapaksa College, Sri Lanka",
      "duration": "2008 - 2010"
    },
    {
      "degree": "General Certificate of Education",
      "field_of_study": "A-Level (Physical Sciences)",
      "type": "Private Candidate"
    }
  ],
  "professional_experience": [
    {
      "position": "Senior Software Engineer",
      "company": "BISTEC Global Services",
      "duration": "2023 - Present",
      "achievements": ["Lead development of multiple web applications", "Implemented Agile methodologies"]
    },
    {
      "position": "Associate Software Engineer",
      "company": "BISTEC Global Services",
      "duration": "2022 - 2023",
      "achievements": ["Assisted in backend development", "Supported project management tasks"]
    },
    {
      "position": "Software Development Intern",
      "company": "BISTEC Global Services",
      "duration": "2022",
      "achievements": ["Gained practical experience in software development"]
    },
    {
      "position": "Freelance Web Developer",
      "duration": "2020 - 2022",
      "achievements": ["Delivered custom web solutions for clients"]
    },
    {
      "position": "Freelance Graphic Designer",
      "duration": "2015 - 2019",
      "achievements": ["Designed branding materials for various clients"]
    }
  ],
  "skills": {
    "programming_languages": {
      "JavaScript": "Advanced",
      "Python": "Advanced",
      "C#": "Intermediate",
      "Java": "Intermediate",
      "PHP": "Intermediate"
    },
    "web_development": {
      "HTML": "Advanced",
      "CSS": "Advanced",
      "React": "Intermediate",
      ".NET": "Intermediate",
      "ASP.NET Core": "Intermediate"
    },
    "databases": {
      "MySQL": "Intermediate",
      "SQL": "Intermediate"
    }
  },
  "coding_behavior": {
    "preferred_language": "Python",
    "development_methodologies": ["Agile", "Scrum"],
    "collaboration_tools": ["Git", "Jira", "Slack"]
  },
  "writing_topics": [
    {
      "title": "DApp development with Truffle on Ethereum Blockchain for absolute beginners",
      "publication_date": "December 19, 2022",
      "style": "Tutorial"
    },
    {
      "title": "12 Rules for RDBMS by E.F. Codd",
      "publication_date": "January 21, 2023",
      "style": "Analytical"
    }
  ],
  "projects": [],
  "publications": []
}
```

In [69]:
import json

# Extract the raw JSON content from the results
json_content = result.raw.replace('```json\n', '').replace('\n```', '')

# Parse and save as JSON file
with open('gap2.json', 'w') as f:
    json.dump(json.loads(json_content), f, indent=2)
